480 Term Project: Autograding essays based on prompts and responses

In [50]:
import nltk
import sklearn
from sklearn import model_selection
from sklearn.preprocessing import minmax_scale, StandardScaler
from sklearn.metrics import f1_score
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'say'])
#anonymous_words = ["@PERSON", "@ORGANIZATION", "@LOCATION", "@DATE", "@TIME", "@MONEY", 
#                   "@PERCENT","@MONTH","@EMAIL","@NUM","@CAPS","@DR","@CITY","@STATE"]

In [51]:
score_data = pd.read_csv('training_set_rel3.tsv', sep='\t',encoding='mac_roman')
scores_df = pd.DataFrame(data=score_data)

In [52]:
# Set essays in set 2 to be mean of domain1_score and domain2_score
scores_df['essay_score'] = ((scores_df['domain1_score'] + scores_df['domain2_score'])/2)
scores_df['essay_score'] = scores_df['essay_score'].fillna(scores_df['domain1_score'])
scores_df = scores_df.drop(['domain1_score', 'domain2_score'], axis=1)

# Split data into testing and training sets
train_data, test_data = model_selection.train_test_split(scores_df, test_size=0.2)

# Group by essay_set (1-8) and min-max scale essay_score within each group 
def scale_essay_scores(df):
    df['essay_score'] = minmax_scale(df['essay_score'])
    return df

scaled_scores_df = train_data.groupby('essay_set').apply(lambda x : scale_essay_scores(x)).reset_index()
scaled_scores_df = scaled_scores_df.drop('index', axis=1)

y_train = scaled_scores_df['essay_score']
X_train = scaled_scores_df.drop('essay_score', axis=1)

prompt_data = pd.read_csv('training_set_rel3_prompts.csv')
prompts_df = pd.DataFrame(data=prompt_data)

In [53]:
data = X_train['essay'].values.tolist()
prompt_data = prompts_df['prompt'].values.tolist()
passage_data = prompts_df['source_essay'].dropna().values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
prompt_words = list(sent_to_words(prompt_data))
passage_words = list(sent_to_words(passage_data))

In [54]:
# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

prompt_bigram = gensim.models.Phrases(prompt_words, min_count=5, threshold=100) # higher threshold fewer phrases.

passage_bigram = gensim.models.Phrases(passage_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

prompt_bigram_mod = gensim.models.phrases.Phraser(prompt_bigram)

passage_bigram_mod = gensim.models.phrases.Phraser(passage_bigram)

# Helpers for stopwords, bigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def not_anonymous_term(word):
#     for term in anonymous_words:
#         term_regex = re.compile('/{term}/gi')
#         if word.match(term):
#             return false
#     return true

def make_bigrams(texts, text_type):
    if text_type == 'essay':
        return [bigram_mod[doc] for doc in texts]
    elif text_type == 'prompt':
        return [prompt_bigram_mod[doc] for doc in texts]
    elif text_type == 'passage':
        return [passage_bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [38]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
prompt_words_nostops = remove_stopwords(prompt_words)
passage_words_nostops = remove_stopwords(passage_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops, 'essay')
prompt_words_bigrams = make_bigrams(prompt_words_nostops, 'prompt')
passage_words_bigrams = make_bigrams(passage_words_nostops, 'passage')

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
from spacy.lang.en.examples import sentences 

import spacy
nlp=spacy.load('en_core_web_sm')

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
prompt_data_lemmatized = lemmatization(prompt_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
passage_data_lemmatized = lemmatization(passage_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
id2word.add_documents(prompt_data_lemmatized)
id2word.add_documents(passage_data_lemmatized)

# Create Corpus
texts = data_lemmatized + prompt_data_lemmatized + passage_data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [16]:
#LDA Model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus] 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [17]:
# Convert topics into feature vectors (adapted from https://gist.github.com/marcmuon/66300292d428c4ae73645b7598d0dbac#file-get_features-py)
train_ft_vecs = []
for i in range(len(train_data)):
    top_topics = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(3)]
    train_ft_vecs.append(topic_vec)

In [21]:
# Plug ft vecs and essay_score labels into different shallow regressors (sgd and dec tree for now) and evaluate by r2 score

kfold = model_selection.KFold(n_splits=5)
X_train = np.array(train_ft_vecs)

cv_logreg, cv_dtreereg = [], []
for train_i, val_i in kfold.split(X_train, y_train):
    X_tr, y_tr = X_train[train_i], y_train[train_i]
    X_val, y_val = X_train[val_i], y_train[val_i]
    
    # Scale data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_tr)
    X_val_scale = scaler.transform(X_val)
    
    logreg_model = SGDRegressor().fit(X_train_scale, y_tr)
    
    y_pred = logreg_model.score(X_val_scale, y_val)
    cv_logreg.append(y_pred)
    
    dtreereg_model = DecisionTreeRegressor(max_depth=5).fit(X_train_scale, y_tr)
    
    y_pred = dtreereg_model.score(X_val_scale, y_val)
    cv_dtreereg.append(y_pred)

print(f'Decision Tree Regressor R2 score: {np.mean(cv_dtreereg):.3f}')
print(f'SGD Regressor R2 score: {np.mean(cv_logreg):.3f}')

Decision Tree Regressor R2 score: 0.383
SGD Regressor R2 score: 0.033


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None.

In [27]:
cv_svm=[]
    
svm_model = SVR(kernel='rbf').fit(X_train_scale, y_tr)
    
y_pred = svm_model.score(X_val_scale, y_val)
cv_svm.append(y_pred)
print(f'SVM R2 score: {np.mean(cv_svm):.3f}')

SVM R2 score: 0.037


In [32]:
cv_dtreeclass = []
dtreeclass_model = DecisionTreeClassifier().fit(X_train_scale, y_tr)
    
y_pred = dtreeclass_model.score(X_val_scale, y_val)
cv_dtreeclass.append(y_pred)

ValueError: Unknown label type: 'continuous'